In [2]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import unidecode
from collections import Counter

import plotly.express as px
import plotly
import plotly.graph_objects as go


# Load data

In [3]:
path = 'Data/' ## Path Solene


In [27]:
df_montpellier = pd.read_csv(os.path.join(path, 'df_montpellier_clean.csv'), sep=",")

df_rennes= pd.read_csv(os.path.join(path, 'df_rennes_clean.csv'), sep=",")

df_paris = pd.read_csv(os.path.join(path, 'df_paris_pour_compar.csv'), sep=",")

# assembler en dataframe

In [28]:
var_cols = ['Technique & Recherche',
       'Architecture & urbanisme', 'Social', 'Education & formation',
       'Culture & Arts', 'Humanitaire', 'Communication & média',
       'Environnement & écologie', 'Aides aux associations', 'Sport', 'Santé',
       'Relations internationales', 'Idée & opinion', 'Loisirs',
       'Déplacements et transports', 'Tourisme', 'Mémoire',
       'Défense des droits et des intérêts', 'Economie', 'Emploi',
       'Précarité & Exclusion', 'Vie et animation locale']

kepp_cols = ['montant','Technique & Recherche',
       'Architecture & urbanisme', 'Social', 'Education & formation',
       'Culture & Arts', 'Humanitaire', 'Communication & média',
       'Environnement & écologie', 'Aides aux associations', 'Sport', 'Santé',
       'Relations internationales', 'Idée & opinion', 'Loisirs',
       'Déplacements et transports', 'Tourisme', 'Mémoire',
       'Défense des droits et des intérêts', 'Economie', 'Emploi',
       'Précarité & Exclusion', 'Vie et animation locale']

def transformer_df(df_subventions,newname):

    df = df_subventions.copy()
    df_subventions=df[kepp_cols]
    df_subventions[var_cols] = df_subventions[var_cols].div(df_subventions[var_cols].sum(axis=1), axis=0)

    df_subventions[var_cols] = df_subventions[var_cols].multiply(df_subventions["montant"], axis="index")

    final = pd.DataFrame(df_subventions[var_cols].sum())

    final= (100. * final / final.sum()).round(2)

    final=final.rename(columns={0: 'part des subventions (%)'})
    
    final["ville"]=newname

    return final

In [29]:
transformer_df(df_rennes,"Rennes")

,part des subventions (%),ville
Technique & Recherche,0.00,Rennes
Architecture & urbanisme,0.85,Rennes
Social,34.20,Rennes
Education & formation,26.94,Rennes
Culture & Arts,23.03,Rennes
Humanitaire,0.00,Rennes
Communication & média,0.02,Rennes
Environnement & écologie,0.17,Rennes
Aides aux associations,0.36,Rennes
Sport,5.68,Rennes


In [30]:
df_rennes_transf = transformer_df(df_rennes,"Rennes")
df_montpellier_transf = transformer_df(df_montpellier,"Montpellier")
df_paris_transf = transformer_df(df_paris,"Paris")

In [31]:
final = df_rennes_transf.append(df_montpellier_transf).append(df_paris_transf)

final = final.reset_index()

final = final.rename(columns={'index': "activité de l'association"})

In [34]:
# fig = px.bar(final, x="index",y=0)
# fig.update_xaxes(categoryorder = 'total descending')
# fig.show()

In [35]:
final.columns

Index(['activité de l'association', 'part des subventions (%)', 'ville'], dtype='object')

In [57]:
import plotly.express as px
df = px.data.tips()
fig = px.bar(final, y="activité de l'association", x="part des subventions (%)", color='ville', barmode='group',orientation='h',width=700, height=700).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.update_yaxes(categoryorder = 'total ascending')
fig.show()

In [58]:
plotly.offline.plot(fig, filename='website/comparaison_villes_full.html')

'website/comparaison_villes.html'

# filter small activities

In [79]:
keep_vars = ['Social','Sport', 'Santé','Vie et animation locale','Education & formation',
       'Culture & Arts','autres (écologie, mémoire, précarité...)']

autres = ['Technique & Recherche',
       'Architecture & urbanisme',  
        'Humanitaire', 'Communication & média',
       'Environnement & écologie', 'Aides aux associations',
       'Relations internationales', 'Idée & opinion', 'Loisirs',
       'Déplacements et transports', 'Tourisme', 'Mémoire',
       'Défense des droits et des intérêts', 'Economie', 'Emploi',
       'Précarité & Exclusion', ]

keep_cols = ['montant','autres (écologie, mémoire, précarité...)','Social','Sport', 'Santé','Vie et animation locale','Education & formation',
       'Culture & Arts']

def transformer_df_filter(df_subventions,newname):

    df = df_subventions.copy()
    
    df["autres (écologie, mémoire, précarité...)"] = df[autres].sum(axis=1)
    
    df_subventions=df[keep_cols]
    
    df_subventions[keep_vars] = df_subventions[keep_vars].div(df_subventions[keep_vars].sum(axis=1), axis=0)

    df_subventions[keep_vars] = df_subventions[keep_vars].multiply(df_subventions["montant"], axis="index")

    final = pd.DataFrame(df_subventions[keep_vars].sum())

    final= (100. * final / final.sum()).round(2)

    final=final.rename(columns={0: 'part des subventions (%)'})
    
    final["ville"]=newname

    return final

In [80]:
df_rennes_trans_filt = transformer_df_filter(df_rennes,"Rennes")
df_montpellier_transf_filt = transformer_df_filter(df_montpellier,"Montpellier")
df_paris_transf_filt = transformer_df_filter(df_paris,"Paris")

In [106]:
final = df_rennes_trans_filt.append(df_paris_transf_filt).append(df_montpellier_transf_filt)

final = final.reset_index()

final = final.rename(columns={'index': "activité de l'association"})

In [107]:
final

,activité de l'association,part des subventions (%),ville
0,Social,34.20,Rennes
1,Sport,5.68,Rennes
2,Santé,1.11,Rennes
3,Vie et animation locale,2.36,Rennes
4,Education & formation,26.94,Rennes
5,Culture & Arts,23.03,Rennes
6,"autres (écologie, mémoire, précarité...)",6.68,Rennes
7,Social,25.79,Paris
8,Sport,2.42,Paris
9,Santé,6.63,Paris


In [109]:
import plotly.express as px
df = px.data.tips()

fig = px.bar(final,
             y="activité de l'association", 
             x="part des subventions (%)",
             color='ville',
             barmode='group',
             orientation='h',
             width=700,
             height=700,
            color_discrete_sequence=['#31256d', '#69d8c4','#698ed8'],

            ).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))

fig.update_yaxes(categoryorder = 'total ascending')
fig.show()

In [110]:
plotly.offline.plot(fig, filename='website/comparaison_villes_reduit.html')

'website/comparaison_villes_reduit.html'